<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/Processing_pcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Processing pcap__*

Wireshark and other tools like Network Miner are great for interactively exploring packet capture files, but at times you'll want to slice and dice pcap files using Python and Scapy. Some great use cases are generating fuzzing test cases based on captured network traffic or even something as simple as replaying traffic that you have previously captured.

